# Imports

In [1]:
import sys
sys.path.append('../')
from datetime import datetime
import pandas as pd

from run_all.main_preprocess import load_data, add_features
from utilities.utilities import get_latest_file

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

# Preprocess

In [ ]:
%%time
# Load original sources and combine to one DataFrame
df_dataset_WMO = load_data()

In [ ]:
%%time
# Feature engineering to get more features
df_dataset_WMO_with_features = add_features(df_dataset_WMO)

## Optional: Write temporary result

In [4]:
# suffix_datetime = datetime.strftime(datetime.now(), format='%Y%m%d%H%M')

# df_dataset_WMO_with_features.to_parquet(f'../../data/df_preprocess_WMO_{suffix_datetime}.parquet.gzip',
#               compression='gzip')

# Train

## Optional: Load previous dataset

In [5]:
## Continue with loaded data from preprocess
#df = df_dataset_WMO_with_features.copy()

# ## HARDCODED
# datapath = '../../data/'
# filename = 'df_preprocess_WMO_202103211137.parquet.gzip'
# df = pd.read_parquet(datapath + filename)

# ## SELECT LAST FILE
datapath = '../../data/'
df = get_latest_file(filename_str_contains='df_prep_for_train_WMO', datapath=datapath, filetype='parquet')

In [6]:
df.info(verbose = True)

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 18006 entries, ('WK000300', '2019') to ('WK198703', '2017')
Data columns (total 102 columns):
 #   Column                                          Dtype  
---  ------                                          -----  
 0   perioden                                        object 
 1   wmoclienten                                     float64
 2   wmoclientenper1000inwoners                      float32
 3   periodennum                                     int32  
 4   aantalinkomensontvangers                        float64
 5   aantalinwoners                                  float64
 6   actieven1575jaar                                float64
 7   afstandtotgrotesupermarkt                       float64
 8   afstandtothuisartsenpraktijk                    float64
 9   afstandtotkinderdagverblijf                     float64
 10  afstandtotschool                                float64
 11  alandbouwbosbouwenvisserij                      float64
 1

## Train model

### Train imports

In [7]:
import pandas as pd
import numpy as np
from datetime import datetime
import pickle

from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split, cross_val_score, RepeatedKFold, GridSearchCV, cross_validate, KFold, cross_val_score
from sklearn.linear_model import LogisticRegression, LinearRegression, Ridge, Lasso, LassoCV, ElasticNet, BayesianRidge
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer, make_column_transformer
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.svm import SVR
from sklearn.impute import SimpleImputer
from sklearn.neighbors import KNeighborsRegressor
from sklearn.decomposition import PCA
from sklearn.ensemble import RandomForestRegressor

from xgboost import XGBRegressor, XGBClassifier

from ColumnSelector import ColumnSelector

# instellingen voor panda weergave aanpassen
pd.set_option('display.max_rows', 500) # alle rijen tonen
pd.set_option('display.max_columns', 500) # alle kolommen tonen
pd.set_option('display.width', 1000) # kolombreedte
pd.set_option("display.precision", 2)     # precisie van de kolommen aanpassen
pd.set_option('display.float_format', lambda x: '{:.3f}'.format(x)) # floats output tot 3 decimalen

### Settings

In [52]:
## Dataframe parameters
# locatie van dataset 
DF_LOCATION = 'C:/_NoBackup/Git/__JADS/WMO_execute_group_project/data/df_dataset_WMO.parquet.gzip'
# Location all data
datapath = '../../data/'
# manier van laden dataset. Bijvoorbeeld read_parquet of read_csv
DF_READ = pd.read_parquet

## X & Y parameters
# de kolommen die uit de X dataset moeten worden gehaald. Dat is in ieder geval de y en eventueel nog meer kolommen.
# X_DROP_VALUES = ['wmoclienten', 'eenpersoonshuishoudens', 'huishoudenszonderkinderen', 'huishoudensmetkinderen']
X_DROP_VALUES = ['wmoclienten', 'percentagewmoclienten','wmoclientenper1000inwoners','perioden']
# de kolom die wordt gebruikt als y value
Y_VALUE = ['wmoclientenper1000inwoners']
# test size voor de train/test split
TEST_SIZE = 0.3
# random state voor de train/test split. Bijvoorbeeld random_state = 42 als vaste seed voor reproduceerbaarheid
RANDOM_STATE = 42

## Pipeline parameters
# strategy en waarde om te vullen bij lege categorische kolommen
NAN_VALUES_CAT_STRATEGY = 'constant'
NAN_VALUES_CAT_VALUES = 'Missing'
# waarden om in te vullen bij lege numerieke kolommen. Bijvoorbeeld mean of median
NAN_VALUES_NUM_STRATEGY = 'mean'
# 
#COLS_SELECT = ['aantalinwoners', 'mannen', 'vrouwen', 'k0tot15jaar'
#               , 'k15tot25jaar', 'k25tot45jaar', 'k45tot65jaar', 'k65jaarofouder', 'gescheiden'
#               , 'verweduwd', 'westerstotaal', 'sterftetotaal', 'gemiddeldehuishoudensgrootte'
#               , 'gemiddeldewoningwaarde', 'koopwoningen', 'huurwoningentotaal', 'inbezitwoningcorporatie'
#               , 'gemiddeldinkomenperinkomensontvanger', 'k40personenmetlaagsteinkomen', 'k20personenmethoogsteinkomen'
#               , 'actieven1575jaar', 'k40huishoudensmetlaagsteinkomen', 'k20huishoudensmethoogsteinkomen'
#               , 'huishoudensmeteenlaaginkomen', 'personenpersoortuitkeringaow', 'rucultuurrecreatieoverigediensten'
#               , 'personenautosperhuishouden', 'matevanstedelijkheid']
COLS_SELECT = None

## Model parameters
# manier van cross validate in de modellen. Bijvoorbeeld 10 of RepeatedKFold(n_splits=30, n_repeats=5, random_state=1)
CROSS_VALIDATE = 2
# manier van scoren in de modellen
MODEL_SCORING = 'neg_mean_squared_error'

## Scoring parameters
# Deze kunnen we later toevoegen als we meerdere manieren van scoren hebben. Dus niet alleen maar de RSME

### Functions

In [9]:
# functie maken om op basis van de cv scores, het beste RMSE model te selecteren 
def get_best_model_rmsle(cv_scores):
    """
    Return best (most conservative) model from cross_validate object.
    
    Uses np.argmax to find bottomright point == largest RMSE
    """
    index = np.argmax(np.sqrt(-cv_scores['train_neg_mean_squared_error']))
    model = cv_scores['estimator'][index]
    rmse = np.sqrt(mean_squared_error(y_test, model.predict(X_test)))
    return (rmse)

### Load data

In [10]:
# ## Done before start of 'Train' chapter
# df = get_latest_file(mypath=datapath)

#### Stappen hieronder mogelijk verplaatsten naar prepare stap, later beoordelen

In [11]:
# droppen van de rijen waar de y_value leeg is, anders kunnen de modellen er niet mee overweg
df.dropna(
    axis=0,
    how='any',
    thresh=None,
    subset=Y_VALUE,
    inplace=True
)

In [12]:
# X en y aanmaken
X = df.drop(X_DROP_VALUES, axis=1)
# y = df[Y_VALUE]*100 # 0.01 -> 1.0 percentage
y = df[Y_VALUE] # 0.01 -> 1.0 percentage

In [13]:
# splitsen van X en y in train/test. 
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = TEST_SIZE, random_state = RANDOM_STATE)

In [14]:
# splitsen van X_train in categorische en numerieke kolommen, om apart te kunnen transformeren
cat_cols = X_train.select_dtypes(include=['category']).columns
num_cols = X_train.select_dtypes(include=['int64','float64','float32','int32']).columns

### Pipelines

In [15]:
# pipelines (pl) maken voor imputing, scaling en OneHotEncoding per datatype 

# categorie met waarde die is gegeven aan "MISSING" toevoegen
for col in cat_cols:
    # need to add category for missings, otherwise error with OneHotEncoding (volgens mij ook met alleen imputing)
    X_train[col].cat.add_categories(NAN_VALUES_CAT_VALUES, inplace=True)
categories = [X_train[col].cat.categories for col in cat_cols]

# pipeline voor categorial datatype
pl_ppc_cat = make_pipeline(
     SimpleImputer(
         missing_values = np.nan
        ,strategy = NAN_VALUES_CAT_STRATEGY
        ,fill_value = NAN_VALUES_CAT_VALUES)
    ,OneHotEncoder(categories=categories)
)

# pipeline voor numeriek datatype
pl_ppc_num = make_pipeline(
      ColumnSelector(cols=COLS_SELECT)
    ,SimpleImputer(
         missing_values = np.nan
        ,strategy = NAN_VALUES_NUM_STRATEGY)
    ,StandardScaler()
    ,PCA() # PCA heeft behoorlijk wat (positieve) invloed op de scores
)

In [16]:
# pipelines maken om de preprocessing van de imputing te combineren
pl_ppc_total = make_column_transformer(
     (pl_ppc_cat, cat_cols)
    ,(pl_ppc_num, num_cols)
    ,remainder = 'drop'
)

## Gridsearch

Het maken van een GridSearch pipeline voor elk model:

In [69]:
# pipeline maken voor Linear Regression
pl_gs_lr = Pipeline(
     [('preprocess', pl_ppc_total),
     ('model_lr',LinearRegression()) 
     ]
)

# pipeline maken voor RidgeRegression
pl_gs_rr = Pipeline(
     [('preprocess', pl_ppc_total),
     ('model_rr',Ridge()) 
     ]
)

# pipeline maken voor Lasso
pl_gs_lasso = Pipeline(
     [('preprocess', pl_ppc_total),
       ('model_lasso',Lasso()) 
     ]
)

# pipeline maken voor KNN
pl_gs_knn = Pipeline(
     [('preprocess', pl_ppc_total),
       ('model_knn', KNeighborsRegressor()) 
     ]
)

# pipeline maken voor SVM
pl_gs_svm = Pipeline(
     [('preprocess', pl_ppc_total),
       ('model_svm', SVR()) 
     ]
)

# pipeline maken voor XGB
pl_gs_xgb = Pipeline(
     [('preprocess', pl_ppc_total),
       ('model_xgb',XGBRegressor()) 
     ]
)

Het zetten van de gridsearch parameters per model

In [70]:
ALPHA = [0.001, 0.01, 0.1, 1, 10, 100, 1000]
NEIGHBORS = [3, 5, 11, 19]
NORMALIZE = [True, False]
KERNEL = ['linear', 'poly', 'rbf', 'sigmoid', 'precomputed']
GAMMA = [0.5, 1, 1.5, 2, 5]

# Parameters Linieare Regression
param_grid_lr = [
    {'model_lr__normalize': NORMALIZE},
]

# Parameters RidgeRegression
param_grid_rr = [
     {'model_rr__alpha': ALPHA},  
  ]

# Parameters KNneighbors
param_grid_knn = [
     {'model_knn__n_neighbors': NEIGHBORS },  
  ]

# Parameters SVM
param_grid_svm = [
     {'model_svm__kernel': KERNEL },  
  ]

# Parameters XGB
param_grid_xgb = [
     {'model_xgb__gamma': GAMMA},  
  ]

In [ ]:
%%time
# Gridsearch LR 
grid_search_lr = GridSearchCV(pl_gs_lr, param_grid_lr, cv=CROSS_VALIDATE,
                           scoring=MODEL_SCORING,
                           return_train_score=True)
grid_search_lr.fit(X_train, y_train)

In [ ]:
%%time
# Gridsearch RR 
grid_search_rr = GridSearchCV(pl_gs_rr, param_grid_rr, cv=CROSS_VALIDATE,
                           scoring=MODEL_SCORING,
                           return_train_score=True)
grid_search_rr.fit(X_train, y_train)

In [60]:
%%time
# Gridsearch KNN 
grid_search_knn = GridSearchCV(pl_gs_knn, param_grid_knn, cv=CROSS_VALIDATE,
                           scoring=MODEL_SCORING,
                           return_train_score=True)
grid_search_knn.fit(X_train, y_train)

CPU times: user 56.3 s, sys: 785 ms, total: 57.1 s
Wall time: 1min 1s


GridSearchCV(cv=2,
             estimator=Pipeline(steps=[('preprocess',
                                        ColumnTransformer(transformers=[('pipeline-1',
                                                                         Pipeline(steps=[('simpleimputer',
                                                                                          SimpleImputer(fill_value='Missing',
                                                                                                        strategy='constant')),
                                                                                         ('onehotencoder',
                                                                                          OneHotEncoder(categories=[]))]),
                                                                         Index([], dtype='object')),
                                                                        ('pipeline-2',
                                                              

In [67]:
%%time
# Gridsearch SVM 
grid_search_svm = GridSearchCV(pl_gs_svm, param_grid_svm, cv=CROSS_VALIDATE,
                           scoring=MODEL_SCORING,
                           return_train_score=True)
grid_search_svm.fit(X_train, y_train)

/Users/nickneppelenbroek/opt/anaconda3/lib/python3.8/site-packages/sklearn/utils/validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)
/Users/nickneppelenbroek/opt/anaconda3/lib/python3.8/site-packages/sklearn/utils/validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)
/Users/nickneppelenbroek/opt/anaconda3/lib/python3.8/site-packages/sklearn/utils/validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)
/Users/nickneppelenbroek/opt/anaconda3/lib/python3.8/site-packages/sklearn/utils/validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was e

CPU times: user 1min 53s, sys: 1.79 s, total: 1min 54s
Wall time: 2min 4s


GridSearchCV(cv=2,
             estimator=Pipeline(steps=[('preprocess',
                                        ColumnTransformer(transformers=[('pipeline-1',
                                                                         Pipeline(steps=[('simpleimputer',
                                                                                          SimpleImputer(fill_value='Missing',
                                                                                                        strategy='constant')),
                                                                                         ('onehotencoder',
                                                                                          OneHotEncoder(categories=[]))]),
                                                                         Index([], dtype='object')),
                                                                        ('pipeline-2',
                                                              

In [71]:
%%time
# Gridsearch XGB
grid_search_xgb = GridSearchCV(pl_gs_xgb, param_grid_xgb, cv=CROSS_VALIDATE,
                           scoring=MODEL_SCORING,
                           return_train_score=True)
grid_search_xgb.fit(X_train, y_train)

CPU times: user 3min 42s, sys: 2.6 s, total: 3min 44s
Wall time: 1min 29s


GridSearchCV(cv=2,
             estimator=Pipeline(steps=[('preprocess',
                                        ColumnTransformer(transformers=[('pipeline-1',
                                                                         Pipeline(steps=[('simpleimputer',
                                                                                          SimpleImputer(fill_value='Missing',
                                                                                                        strategy='constant')),
                                                                                         ('onehotencoder',
                                                                                          OneHotEncoder(categories=[]))]),
                                                                         Index([], dtype='object')),
                                                                        ('pipeline-2',
                                                              

In [ ]:
grid_search.best_params_

In [ ]:
pd_grid_search = pd.DataFrame(grid_search.cv_results_)
pd_grid_search.head(100)

In [ ]:
grid_search.best_estimator_

## Train pipelines maken

In [ ]:
# pipeline maken voor LinearRegression 
pl_lr = make_pipeline(
     pl_ppc_total
    ,LinearRegression()
)

In [ ]:
# pipeline maken voor RidgeRegression 
pl_rr = make_pipeline(
     pl_ppc_total
    ,Ridge()
)

In [ ]:
# pipeline maken voor Lasso
pl_lasso = make_pipeline(
      pl_ppc_total
     ,Lasso(alpha=0.001)
)

In [ ]:
# pipeline maken voor KNN
pl_knn = make_pipeline(
      pl_ppc_total
     ,KNeighborsRegressor()
)

In [ ]:
# pipeline maken voor SVM
pl_svm = make_pipeline(
      pl_ppc_total
     ,SVR()
)

In [ ]:
# pipeline maken voor XGB
pl_xgb = make_pipeline(
      pl_ppc_total
     ,XGBRegressor()
) 

### Train

##### Lineair Regression

In [ ]:
# scores voor LR berekenen
lr_scores = cross_validate(
    pl_lr, X_train, y_train,
    cv = CROSS_VALIDATE,
    scoring=([MODEL_SCORING]),
    return_train_score=True,
    return_estimator=True,
)

##### Ridge Regression

In [ ]:
rr_scores = cross_validate(
    pl_rr, X_train, y_train,
    cv = CROSS_VALIDATE,
    scoring = ([MODEL_SCORING]),
    return_train_score=True,
    return_estimator=True,
)

##### Lasso

In [ ]:
lasso_scores = cross_validate(
    pl_lasso, X_train, y_train,
    cv = CROSS_VALIDATE,
    scoring = ([MODEL_SCORING]),
    return_train_score=True,
    return_estimator=True,
)

##### K Nearest Neighbor

In [ ]:
knn_scores = cross_validate(
    pl_knn, X_train, y_train,
    cv = CROSS_VALIDATE,
    scoring = ([MODEL_SCORING]),
    return_train_score=True,
    return_estimator=True,
)

##### Support Vector Machines

In [ ]:
svm_scores = cross_validate(
    pl_svm, X_train, y_train,
    cv = CROSS_VALIDATE,
    scoring = ([MODEL_SCORING]),
    return_train_score=True,
    return_estimator=True,
)

##### XGBoost

In [ ]:
xgb_scores = cross_validate(
    pl_xgb, X_train, y_train,
    cv = CROSS_VALIDATE,
    scoring = ([MODEL_SCORING]),
    return_train_score=True,
    return_estimator=True,
)

### Evaluate

#### Calculate scores

In [ ]:
# toevoegen van de beste scores, per model, aan een lijst
scores = []
scores.append(('Linear Regression',  get_best_model_rmsle(lr_scores)))
scores.append(('Ridge Regression',  get_best_model_rmsle(rr_scores)))
scores.append(('Lasso', get_best_model_rmsle(lasso_scores)))
scores.append(('K Nearest Neighbor', get_best_model_rmsle(knn_scores)))
scores.append(('Support Vector Machines', get_best_model_rmsle(svm_scores)))
scores.append(('XGBoost', get_best_model_rmsle(xgb_scores)))

In [ ]:
scores.append(('Lasso', get_best_model_rmsle(lasso_scores)))

In [99]:
# maken van een dataframe met daarin de gesorteerde scores per model
scores = pd.DataFrame(scores)
scores.columns = ['Algorithm', 'RMSE'] 
scores['RMSE'] = scores['RMSE'].map('{:,.10f}'.format)
scores = scores.sort_values('RMSE', ascending=True)
scores = scores.reset_index(drop=True)
scores

,Algorithm,RMSE
0,Lasso,0.0020636175


In [93]:
# and the winner is...
print(f"Het algoritme met de laagste RMSE is:\n\n{scores.iloc[0,0]}, met een RMSE van {scores.iloc[0,1]}")

Het algoritme met de laagste RMSE is:

Lasso, met een RMSE van 0.0114301235


#### Feature importance

In [ ]:
## TODO

### Save model

In [ ]:
# Temporary: select best model (needs function)
BEST_SCORE = lasso_scores
index = np.argmax(np.sqrt(-BEST_SCORE['train_neg_mean_squared_error']))
model = BEST_SCORE['estimator'][index]

In [ ]:
suffix_datetime = datetime.strftime(datetime.now(), format='%Y%m%d%H%M')
filename = f'../../data/best_model_{suffix_datetime}.pickle'

pickle.dump(model, open(filename, 'wb'))

#### Test load model

In [ ]:
loaded_model = get_latest_file(filename_str_contains='best_model_', datapath=datapath, filetype='pickle')

In [ ]:
result = loaded_model.score(X_test, y_test)
print(result)

In [ ]:
loaded_model.predict(X_test)